# Data Science w chmurze: Metoda "Azure ML SDK"

## Wprowadzenie

W tym notatniku nauczymy się, jak używać Azure ML SDK do trenowania, wdrażania i korzystania z modelu za pomocą Azure ML.

Wymagania wstępne:
1. Utworzyłeś przestrzeń roboczą Azure ML.
2. Załadowałeś [zbiór danych dotyczący niewydolności serca](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) do Azure ML.
3. Przesłałeś ten notatnik do Azure ML Studio.

Kolejne kroki to:

1. Utworzenie eksperymentu w istniejącej przestrzeni roboczej.
2. Utworzenie klastra obliczeniowego.
3. Załadowanie zbioru danych.
4. Konfiguracja AutoML za pomocą AutoMLConfig.
5. Uruchomienie eksperymentu AutoML.
6. Analiza wyników i wybór najlepszego modelu.
7. Rejestracja najlepszego modelu.
8. Wdrożenie najlepszego modelu.
9. Korzystanie z punktu końcowego.

## Importy specyficzne dla Azure Machine Learning SDK


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Inicjalizacja przestrzeni roboczej
Zainicjalizuj obiekt przestrzeni roboczej na podstawie zapisanej konfiguracji. Upewnij się, że plik konfiguracyjny znajduje się w .\config.json


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Utwórz eksperyment Azure ML

Stwórzmy eksperyment o nazwie 'aml-experiment' w przestrzeni roboczej, którą właśnie zainicjowaliśmy.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Utwórz klaster obliczeniowy
Będziesz musiał utworzyć [cel obliczeniowy](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) dla swojego uruchomienia AutoML.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Dane
Upewnij się, że przesłałeś zestaw danych do Azure ML i że klucz ma taką samą nazwę jak zestaw danych.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## Konfiguracja AutoML


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## Uruchomienie AutoML


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Wdrażanie najlepszego modelu

Uruchom poniższy kod, aby wdrożyć najlepszy model. Stan wdrożenia możesz sprawdzić w portalu Azure ML. Ten krok może zająć kilka minut.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Korzystanie z punktu końcowego
Możesz dodać dane wejściowe do poniższego przykładowego wejścia.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczeniowej AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby zapewnić dokładność, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za wiarygodne źródło. W przypadku informacji krytycznych zaleca się skorzystanie z profesjonalnego tłumaczenia wykonanego przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z korzystania z tego tłumaczenia.
